In [1]:
import numpy as np
import pandas as pd

import twint

# Fixes runtime errors with twint
import nest_asyncio
nest_asyncio.apply()

import matplotlib
%matplotlib inline
import seaborn as sns; sns.set_style('ticks')

import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
import re
import string
import emoji

import gzip
import pickle

from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers

%load_ext autoreload

%autoreload

from functions import *

Using TensorFlow backend.
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/learn-env/lib/p

In [2]:
pd.set_option('max_colwidth', 150)

# Train Test Split

In [3]:
with gzip.open('Datasets/features_df.pkl', 'rb') as handle:
    features_df = pickle.load(handle)

In [4]:
df = pd.read_csv('Datasets/5k_tweets_lda_10.csv', index_col=0)
# df.head()

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,conversation_id,created_at,date,tweet,hashtags,user_id,username,name,link,...,clean_tweets,lda_1,lda_2,lda_3,lda_4,lda_5,lda_6,lda_7,lda_8,lda_9
0,1.212523e+18,1.212523e+18,1.577923e+12,2020-01-01,"https://mltshp.com/p/1HLSB ""This is the death mask of Thomas, an unknown peasant that died in a river.""",[],8.431951e+17,Best_of_MLTSHP,MLTSHP,https://twitter.com/Best_of_MLTSHP/status/1212523477896294400,...,death thomas unknown peasant died river,0,0,0,0,0,1,0,0,0
1,1.212518e+18,1.212518e+18,1.577922e+12,2020-01-01,Ready to heard this? Homemade hair treatment and ann cherry dead sea mud mask... Fucking putas swear they are more woman than me.... Bitch shut th...,[],1.044739e+18,lumora_lu,MiLu,https://twitter.com/lumora_lu/status/1212518043353530368,...,ready heard homemade hair treatment ann cherry dead sea mud fucking putas swear woman bitch shut fuck,0,0,0,0,1,0,0,0,0
2,1.212518e+18,1.212518e+18,1.577922e+12,2020-01-01,"We all know what a joke #CCPChina is when it comes to epidemic outbreak,think of how they stayed silent about #SARS.\nDear world,stay away from #H...","['#ccpchina', '#sars', '#hk', '#hkers']",9.630325e+17,odiecher,odiecher,https://twitter.com/odiecher/status/1212517557690748928,...,joke #ccpchina come epidemic outbreakthink stayed silent #sars dear worldstay away #hk im sure infected amongst already dear fellow #hkers facewit...,1,0,0,0,0,0,0,0,0
3,1.212516e+18,1.200523e+18,1.577921e+12,2020-01-01,Thought death masks gave you the horn freak,[],2.819219e+09,Hevysmoker,Ashley Collins,https://twitter.com/Hevysmoker/status/1212515585562726401,...,thought death gave horn freak,0,0,0,0,0,0,0,1,0
4,1.212510e+18,1.212391e+18,1.577920e+12,2020-01-01,"Interesting cult of death mask he’s wearing, 🤔\nelectronically posing as he did as of the CIA while Brennan was Chief. 🧐",[],2.361956e+08,FaithR8s,❌Patriot Fan🇺🇸🗽☕️🎶❌,https://twitter.com/FaithR8s/status/1212510409858670593,...,interesting cult death thinkingface electronically posing cia brennan chief facewithmonocle,0,0,0,0,0,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_df, df.sentiment, test_size=0.25, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [43]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# LSTM

In [ ]:
# Use padding or truncation to make each observation have 400 features
train_features = sequence.pad_sequences(X_train_scaled, maxlen=1000, dtype='float64', truncating='post')
test_features = sequence.pad_sequences(X_test_scaled, maxlen=1000, dtype='float64', truncating='post')

In [52]:
print(X_train_scaled[0])

[0. 0. 0. ... 0. 0. 0.]


In [53]:
# View first observation
test_features[0]

array([0.        , 0.        , 0.        , 0.55      , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [ ]:
# Start neural network
network = Sequential()

# Add an embedding layer
network.add(layers.Embedding(input_dim=X_train.shape[1], output_dim=128))

# Add a long short-term memory layer with 128 units
network.add(layers.LSTM(units=128))

# Add fully connected layer with a softmax activation function
network.add(layers.Dense(units=1, activation='softmax'))

In [ ]:
# Compile neural network
network.compile(loss='binary_crossentropy', # Cross-entropy
                optimizer='Adam', # Adam optimization
                metrics=['accuracy']) # Accuracy performance metric

In [57]:
y_train.head()

36702     negative
268432    positive
474540     neutral
347524    negative
562666    negative
Name: sentiment, dtype: object

In [ ]:
conditions = [
    y_train == 'negative',
    y_train == 'neutral'
]

choices = [0,1]

conditions_test = [
    y_test == 'negative',
    y_test == 'neutral'
]

In [ ]:
y_train_num = pd.DataFrame(np.select(conditions, choices, 2))
y_train_num

In [ ]:
y_test_num = pd.DataFrame(np.select(conditions_test, choices, 2))
y_test_num

In [ ]:
# Train neural network
history = network.fit(train_features, # Features
                      y_train_num, # Target
                      epochs=3, # Number of epochs
                      verbose=1, 
                      batch_size=50000, # Number of observations per batch
                      validation_data=(test_features, y_test_num)) # Data for evaluation

In [ ]:
# create the model
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Tomek/

In [ ]:
from imblearn.under_sampling import TomekLinks

tomek = TomekLinks()
X_res, y_res = tomek.fit_resample(X_train_scaled, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  depr

# Random Forest Classifier

In [ ]:
# random forest
forest_res = RandomForestClassifier(
                                        class_weight='balanced',
                                        random_state=1, n_jobs=-1,
                                        n_estimators=50, max_depth=2)

# fit training data
forest_res.fit(X_train_res, y_train_res)

# predict test dta
y_pred_forest = forest_res.predict(X_test_scaled)

# check accuracy
print('Test Accuracy score: ', accuracy_score(y_test, y_pred_forest))
# check f1
print('Test F1 score: ', f1_score(y_test, y_pred_forest, average='weighted'))
# check recall
print('Test Recall score: ', recall_score(y_test, y_pred_forest, average='weighted'))
# check precision
print('Test Precision score: ', precision_score(y_test, y_pred_forest, average='weighted'))

# SCRAP HEAP

In [126]:
pos = TextBlob(tweets[2])
print(pos.tags)

[('Using', 'VBG'), ('the', 'DT'), ('coverage', 'NN'), ('of', 'IN'), ('his', 'PRP$'), ('yellow', 'JJ'), ('card', 'NN'), ('to', 'TO'), ('mask', 'VB'), ('the', 'DT'), ('fact', 'NN'), ('his', 'PRP$'), ('team', 'NN'), ('lost', 'VBD'), ('to', 'TO'), ('Southampton', 'NNP'), ('ClassicJose', 'NNP'), ('CoverUp', 'NNP')]


In [122]:
for tag in pos.tags:
    if 'mask' in tag[0]:
        print(tag)

('masks', 'NNS')


In [115]:
for tweet in tweets[5:10]:
    pos = TextBlob(tweet)
    print(pos.tags)

[('.My', 'NN'), ('daughter', 'NN'), ('has', 'VBZ'), ('been', 'VBN'), ('sick', 'VBN'), ('with', 'IN'), ('a', 'DT'), ('nasty', 'JJ'), ('cough', 'NN'), ('for', 'IN'), ('weeks', 'NNS'), ('She', 'PRP'), ('ignores', 'VBZ'), ('my', 'PRP$'), ('advice', 'NN'), ('about', 'IN'), ('wearing', 'VBG'), ('a', 'DT'), ('mask', 'NN'), ('Can', 'NNP'), ('any', 'DT'), ('Drs', 'NNP'), ('out', 'IN'), ('there', 'RB'), ('please', 'VB'), ('tweet', 'NN'), ('about', 'IN'), ('the', 'DT'), ('health', 'NN'), ('impacts', 'NNS'), ('of', 'IN'), ('breathing', 'VBG'), ('this', 'DT'), ('toxic', 'NN'), ('air', 'NN'), ('even', 'RB'), ('when', 'WRB'), ('it', 'PRP'), ("'s", 'VBZ'), ('not', 'RB'), ('smelling', 'JJ'), ('strongly', 'RB'), ('of', 'IN'), ('smoke', 'NN'), ('so', 'IN'), ('that', 'IN'), ('I', 'PRP'), ('can', 'MD'), ('show', 'VB'), ('her', 'PRP$'), ('I', 'PRP'), ('am', 'VBP'), ('not', 'RB'), ('full', 'JJ'), ('of', 'IN'), ('shit', 'NN')]
[('😺✏', 'NN'), ('—', 'NNP'), ('Aside', 'NNP'), ('from', 'IN'), ('butt', 'NN'), ('ma

In [24]:
Counter(words).most_common(25)

[('hand', 40134),
 ('need', 39383),
 ('spread', 33418),
 ('protect', 32088),
 ('make', 31605),
 ('help', 30306),
 ('say', 28477),
 ('glove', 27649),
 ('public', 25717),
 ('hospital', 25525),
 ('new', 25340),
 ('time', 25085),
 ('medical', 24652),
 ('china', 24641),
 ('doctor', 24152),
 ('home', 23986),
 ('work', 23382),
 ('day', 22567),
 ('stop', 21958),
 ('trump', 21074),
 ('infected', 21008),
 ('please', 20635),
 ('everyone', 20359),
 ('health', 20312),
 ('death', 20287)]